<a href="https://colab.research.google.com/github/Abhimanyu004/CNN_projects/blob/main/fashion_img_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [8]:
fashion = keras.datasets.fashion_mnist

In [10]:
(train_img, train_labels), (test_img, test_labels) = fashion.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [11]:
train_img = train_img/255.0
test_img = test_img/255.0

In [14]:
train_img.shape

(60000, 28, 28)

In [15]:
train_img = train_img.reshape(len(train_img), 28,28,1)
test_img = test_img.reshape(len(test_img), 28,28,1)

In [16]:
train_img.shape

(60000, 28, 28, 1)

In [17]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('Conv_filter_1', min_value = 32, max_value = 128, step = 16),
          kernal_size = hp.Choice('convo_kernal_1', values = [3,5]),
          activation ='relu',
          input_shape = (28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_filter_2', min_value=32, max_value=64, step=16),
          kernel_size=hp.Choice('conv_kernel_2', values = [3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model